In [ ]:
!pip install coremltools==4.1

     |████████████████████████████████| 3.4 MB 4.6 MB/s 
  Created wheel for attr: filename=attr-0.3.1-py3-none-any.whl size=2457 sha256=abc8292a8d1715cec112697a5812f116e459b44e8379cc183917f62ed544a9e1
  Stored in directory: /root/.cache/pip/wheels/3b/5d/58/41fbe92f47031641008bd8559ee89e58bf0f123f9c18dea1cb
Successfully built attr


In [1]:
%tensorflow_version 2.x

In [2]:
import sys
print(sys.version)

import tensorflow as tf
print(tf.__version__)

import coremltools as ct
print(ct.__version__)

3.7.11 (default, Jul  3 2021, 18:01:19) 
[GCC 7.5.0]
2.5.0


4.1


In [3]:
def convert(keras_model, 
            coreml_file,
            input_name,
            output_name):
  
  # Download class labels (from a separate file)
  import urllib
  label_url = 'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt'
  class_labels = urllib.request.urlopen(label_url).read().splitlines()
  class_labels = class_labels[1:] # remove the first class which is background
  assert len(class_labels) == 1000

  # make sure entries of class_labels are strings
  for i, label in enumerate(class_labels):
    if isinstance(label, bytes):
      class_labels[i] = label.decode("utf8")

  image_input = ct.ImageType(shape=(1, 224, 224, 3),
                              bias=[-1,-1,-1],
                              scale=1/127)

  # set class labels
  classifier_config = ct.ClassifierConfig(class_labels)

  # Convert the model using the Unified Conversion API
  model = ct.convert(
      keras_model,
      source='tensorflow', 
      inputs=[image_input], 
      classifier_config=classifier_config,
  )

  print(model.input_description, '->', model.output_description)

  # Set feature descriptions (these show up as comments in XCode)
  model.input_description[input_name] = "Input image to be classified"
  model.output_description[output_name] = "Most likely image category"

  # Set model author name
  model.author = 'Converted from TF to Core ML by Anh'

  # Set a version for the model
  model.version = "2021-08-04"

  spec = model._spec

  ct.utils.rename_feature(spec, input_name, "image")
  ct.utils.rename_feature(spec, output_name, "classLabelProbs")

  print(model.input_description, '->', model.output_description)

  model.save(coreml_file)

In [ ]:
mnv2 = tf.keras.applications.MobileNetV2(
    weights="imagenet", 
    input_shape=(224, 224, 3,),
    classes=1000,
)
convert(mnv2, 'MobileNetV2.mlmodel', input_name = 'input_1', output_name = 'classLabel')

Translating MIL ==> MLModel Ops: 100%|██████████| 747/747 [00:00<00:00, 1360.12 ops/s]


Features(input_1) -> Features(Identity,classLabel)
Features(image) -> Features(Identity,classLabelProbs)


In [4]:
mnv3 = tf.keras.applications.MobileNetV3Large(
    weights="imagenet", 
    input_shape=(224, 224, 3,),
    classes=1000,
)
convert(mnv3, 'MobileNetV3Large.mlmodel', input_name = 'input_1', output_name = 'classLabel')

Translating MIL ==> MLModel Ops: 100%|██████████| 1038/1038 [00:00<00:00, 1185.38 ops/s]


Features(input_1) -> Features(Identity,classLabel)
Features(image) -> Features(Identity,classLabelProbs)
